In [1]:
# Celda 1: Imports y configuración
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
import sqlite3
import time
from datetime import datetime, timedelta
import re
import json

# Configuración de paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Importar configuración
from config import APIFY_API_KEY, DB_PATH
from src.utils.database import get_db_connection, insert_dataframe_to_table

# Importar ApifyClient
from apify_client import ApifyClient

# Configuración de Apify
client = ApifyClient(APIFY_API_KEY)

In [2]:
# Configuración ajustable
LIMITE_RESTAURANTES = 2  # Puedes cambiar este número
DELAY_ENTRE_RESTAURANTES = 5  # Segundos de espera

print(f"⚙️  Configuración: {LIMITE_RESTAURANTES} restaurantes, delay {DELAY_ENTRE_RESTAURANTES}s")

⚙️  Configuración: 2 restaurantes, delay 5s


In [3]:
# Celda 2: Crear tabla Reviews definitiva
def create_reviews_table_definitiva():
    """
    Crea la tabla Reviews con los campos que realmente necesitamos
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS Reviews (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        restaurante_id INTEGER,
        tripadvisor_id INTEGER,
        review_id TEXT UNIQUE,
        titulo TEXT,
        contenido TEXT,
        rating INTEGER,
        idioma TEXT,
        fecha_publicacion TEXT,
        fecha_experiencia TEXT,
        tipo_viaje TEXT,
        usuario TEXT,
        usuario_id TEXT,
        contribuciones_usuario INTEGER,
        votos_utiles INTEGER,
        respuesta_empresa TEXT,
        subratings TEXT,
        fotos TEXT,
        fecha_insercion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (restaurante_id) REFERENCES Restaurantes(id)
    );
    """
    
    try:
        cursor.execute("DROP TABLE IF EXISTS Reviews")  # Eliminar la temporal
        cursor.execute(create_table_sql)
        conn.commit()
        print("✅ Tabla 'Reviews' definitiva creada")
    except Exception as e:
        print(f"❌ Error creando tabla Reviews: {e}")
    finally:
        conn.close()

# create_reviews_table_definitiva()

In [4]:
# Celda 3: Obtener restaurantes para escanear
def get_restaurantes_para_escaneo(limite=2):
    """
    Obtiene restaurantes que necesitan ser escaneados (priorizando los más antiguos)
    """
    conn = get_db_connection()
    
    query = """
    SELECT id, tripadvisor_id, nombre, tripadvisor_web, fecha_escaneo_reviews
    FROM Restaurantes 
    WHERE tripadvisor_web IS NOT NULL
    ORDER BY fecha_escaneo_reviews ASC NULLS FIRST, id ASC
    LIMIT ?
    """
    
    try:
        df_restaurantes = pd.read_sql_query(query, conn, params=(limite,))
        print(f"📊 Restaurantes para escanear: {len(df_restaurantes)}")
        return df_restaurantes
    except Exception as e:
        print(f"❌ Error obteniendo restaurantes: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

In [5]:
# Celda 4: Función de scraping CORREGIDA con filtro correcto
def get_reviews_por_restaurante(tripadvisor_web, restaurante_id, tripadvisor_id, nombre_restaurante):
    """
    Obtiene reviews para un restaurante CONFIGURADO para solo ratings 1-2
    """
    print(f"📖 Escaneando: {nombre_restaurante}")
    print(f"🌐 URL: {tripadvisor_web}")
    
    reviews_data = []
    rating_histogram = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    
    # Configuración CORRECTA para el actor - SOLO ratings 1 y 2
    run_input = {
        "startUrls": [{"url": tripadvisor_web}],
        "reviewsLimit": 100,
        "sinceDate": (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
        "reviewRatings": ["ONE", "TWO"],  # ¡ESTE ES EL CAMPO CLAVE!
        "language": "all",
        "proxyConfiguration": {"useApifyProxy": True},
        "includeRestaurants": True
    }
    
    try:
        # Ejecutar el actor
        run = client.actor("maxcopell/tripadvisor-reviews").call(run_input=run_input)
        dataset_items = list(client.dataset(run["defaultDatasetId"]).iterate_items())
        
        print(f"📦 Obtenidas {len(dataset_items)} reviews (solo ratings 1-2)")
        
        # Procesar cada review (todas deberían ser rating 1-2 por el filtro)
        for item in dataset_items:
            rating = item.get('rating', 0)
            
            # Actualizar histograma
            if 1 <= rating <= 5:
                rating_histogram[rating] += 1
            
            # Todas deberían ser < 3 por el filtro, pero por si acaso
            if rating < 3:
                review_data = {
                    'restaurante_id': restaurante_id,
                    'tripadvisor_id': tripadvisor_id,
                    'review_id': item.get('id'),
                    'titulo': item.get('title'),
                    'contenido': item.get('text'),
                    'rating': rating,
                    'idioma': item.get('lang'),
                    'fecha_publicacion': item.get('publishedDate'),
                    'fecha_experiencia': item.get('travelDate'),
                    'tipo_viaje': item.get('tripType'),
                    'usuario': item.get('user', {}).get('name'),
                    'usuario_id': item.get('user', {}).get('userId'),
                    'contribuciones_usuario': item.get('user', {}).get('contributions', {}).get('totalContributions', 0),
                    'votos_utiles': item.get('helpfulVotes', 0),
                    'respuesta_empresa': item.get('ownerResponse', {}).get('text') if item.get('ownerResponse') else None,
                    'subratings': json.dumps(item.get('subratings', [])) if item.get('subratings') else None,
                    'fotos': json.dumps(item.get('photos', [])) if item.get('photos') else None
                }
                reviews_data.append(review_data)
        
        print(f"✅ {len(reviews_data)} reviews negativas (<3 estrellas) procesadas")
        print(f"📊 Histograma de ratings obtenidos: {rating_histogram}")
        
        # Extraer también el ratingHistogram del placeInfo para actualizar Restaurantes
        rating_histogram_completo = extraer_histograma_completo(dataset_items)
        
        return reviews_data, rating_histogram_completo
        
    except Exception as e:
        print(f"❌ Error escaneando {nombre_restaurante}: {e}")
        return [], {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

def extraer_histograma_completo(dataset_items):
    """
    Extrae el histograma completo de ratings del placeInfo
    """
    histograma_completo = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    
    if dataset_items:
        # Tomar el primer item que debería tener el placeInfo completo
        first_item = dataset_items[0]
        place_info = first_item.get('placeInfo', {})
        rating_histogram = place_info.get('ratingHistogram', {})
        
        # Mapear los campos del histograma
        histograma_completo = {
            1: rating_histogram.get('count1', 0),
            2: rating_histogram.get('count2', 0),
            3: rating_histogram.get('count3', 0),
            4: rating_histogram.get('count4', 0),
            5: rating_histogram.get('count5', 0)
        }
    
    return histograma_completo

In [6]:
# Celda 4.1: Función alternativa si el filtro reviewRatings no funciona
def get_reviews_por_restaurante_alternativo(tripadvisor_web, restaurante_id, tripadvisor_id, nombre_restaurante):
    """
    Versión alternativa si el filtro reviewRatings no funciona
    Obtiene TODAS las reviews y filtra localmente
    """
    print(f"📖 Escaneando (alternativo): {nombre_restaurante}")
    
    reviews_data = []
    rating_histogram = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    
    # Configuración para obtener TODAS las reviews
    run_input = {
        "startUrls": [{"url": tripadvisor_web}],
        "reviewsLimit": 100,
        "sinceDate": (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
        "language": "all",
        "proxyConfiguration": {"useApifyProxy": True},
        "includeRestaurants": True
    }
    
    try:
        run = client.actor("maxcopell/tripadvisor-reviews").call(run_input=run_input)
        dataset_items = list(client.dataset(run["defaultDatasetId"]).iterate_items())
        
        print(f"📦 Obtenidas {len(dataset_items)} reviews (todas)")
        
        # Filtrar localmente solo ratings 1-2
        for item in dataset_items:
            rating = item.get('rating', 0)
            
            # Actualizar histograma completo
            if 1 <= rating <= 5:
                rating_histogram[rating] += 1
            
            # Filtrar solo ratings 1-2
            if rating in [1, 2]:
                review_data = {
                    'restaurante_id': restaurante_id,
                    'tripadvisor_id': tripadvisor_id,
                    'review_id': item.get('id'),
                    'titulo': item.get('title'),
                    'contenido': item.get('text'),
                    'rating': rating,
                    'idioma': item.get('lang'),
                    'fecha_publicacion': item.get('publishedDate'),
                    'fecha_experiencia': item.get('travelDate'),
                    'tipo_viaje': item.get('tripType'),
                    'usuario': item.get('user', {}).get('name'),
                    'usuario_id': item.get('user', {}).get('userId'),
                    'contribuciones_usuario': item.get('user', {}).get('contributions', {}).get('totalContributions', 0),
                    'votos_utiles': item.get('helpfulVotes', 0),
                    'respuesta_empresa': item.get('ownerResponse', {}).get('text') if item.get('ownerResponse') else None,
                    'subratings': json.dumps(item.get('subratings', [])) if item.get('subratings') else None,
                    'fotos': json.dumps(item.get('photos', [])) if item.get('photos') else None
                }
                reviews_data.append(review_data)
        
        print(f"✅ {len(reviews_data)} reviews negativas (1-2 estrellas) filtradas")
        print(f"📊 Histograma completo: {rating_histogram}")
        
        return reviews_data, rating_histogram
        
    except Exception as e:
        print(f"❌ Error escaneando {nombre_restaurante}: {e}")
        return [], {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

In [7]:
# Celda 5: Funciones para guardar datos
def guardar_reviews_en_bd(df_reviews):
    """
    Guarda las reviews en la base de datos
    """
    if not df_reviews.empty:
        try:
            conn = get_db_connection()
            df_reviews.to_sql('Reviews', conn, if_exists='append', index=False)
            conn.close()
            print(f"✅ {len(df_reviews)} reviews guardadas en la BD")
        except Exception as e:
            print(f"❌ Error guardando reviews: {e}")
    else:
        print("📭 No hay reviews negativas para guardar")

def actualizar_restaurante_escaneo(restaurante_id, rating_histogram):
    """
    Actualiza la fecha de escaneo y el histograma de ratings para un restaurante
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        update_query = """
        UPDATE Restaurantes 
        SET fecha_escaneo_reviews = CURRENT_TIMESTAMP,
            conteo_rating_1 = ?, conteo_rating_2 = ?, conteo_rating_3 = ?,
            conteo_rating_4 = ?, conteo_rating_5 = ?
        WHERE id = ?
        """
        
        cursor.execute(update_query, (
            rating_histogram[1], rating_histogram[2], rating_histogram[3],
            rating_histogram[4], rating_histogram[5], restaurante_id
        ))
        
        conn.commit()
        conn.close()
        print(f"✅ Restaurante {restaurante_id} actualizado con histograma")
        
    except Exception as e:
        print(f"❌ Error actualizando restaurante: {e}")

In [8]:
# Celda 6: Función principal con selección de método
def ejecutar_escaneo_reviews(limite_restaurantes=2, delay=5, metodo='filtrado'):
    """
    Función principal con opción de método
    metodo: 'filtrado' (usando reviewRatings) o 'alternativo' (filtrando localmente)
    """
    print(f"🎯 INICIANDO ESCANEO DE REVIEWS - Método: {metodo}")
    print("=" * 60)
    
    df_restaurantes = get_restaurantes_para_escaneo(limite_restaurantes)
    
    if df_restaurantes.empty:
        print("🎉 No hay restaurantes para escanear!")
        return
    
    total_reviews_negativas = 0
    
    for i, (_, restaurante) in enumerate(df_restaurantes.iterrows(), 1):
        print(f"\n[{i}/{len(df_restaurantes)}] ", end="")
        
        # Seleccionar método
        if metodo == 'filtrado':
            reviews, rating_histogram = get_reviews_por_restaurante(
                restaurante['tripadvisor_web'],
                restaurante['id'],
                restaurante['tripadvisor_id'],
                restaurante['nombre']
            )
        else:
            reviews, rating_histogram = get_reviews_por_restaurante_alternativo(
                restaurante['tripadvisor_web'],
                restaurante['id'],
                restaurante['tripadvisor_id'],
                restaurante['nombre']
            )
        
        if reviews:
            df_reviews = pd.DataFrame(reviews)
            guardar_reviews_en_bd(df_reviews)
            total_reviews_negativas += len(reviews)
        
        actualizar_restaurante_escaneo(restaurante['id'], rating_histogram)
        
        if i < len(df_restaurantes):
            print(f"⏰ Esperando {delay} segundos...")
            time.sleep(delay)
    
    print(f"\n🎯 ESCANEO COMPLETADO")
    print(f"   • Método usado: {metodo}")
    print(f"   • Restaurantes escaneados: {len(df_restaurantes)}")
    print(f"   • Reviews negativas encontradas: {total_reviews_negativas}")

In [ ]:
# Primero probar con el método filtrado (ideal)
print("🔧 PROBANDO MÉTODO CON FILTRO EN APIFY")
ejecutar_escaneo_reviews(limite_restaurantes=1, delay=3, metodo='filtrado')

In [20]:
# Si no funciona, probar el método alternativo
print("\n\n🔧 PROBANDO MÉTODO ALTERNATIVO (filtro local)")
ejecutar_escaneo_reviews(limite_restaurantes=3, delay=5, metodo='alternativo')



🔧 PROBANDO MÉTODO ALTERNATIVO (filtro local)
🎯 INICIANDO ESCANEO DE REVIEWS - Método: alternativo
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📊 Restaurantes para escanear: 3

[1/3] 📖 Escaneando (alternativo): Restaurant Can Pol dels Angels


[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:47.563Z ACTOR: Pulling Docker image of build YjNFzefvG0FzjjRps from registry.
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:47.565Z ACTOR: Creating Docker container.
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:47.766Z ACTOR: Starting Docker container.
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:47.984Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c npm run start:prod --silent
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:50.200Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:C0Te9GUoVaC86q4RR] -> 2025-10-04T18:36:50.309Z INFO  Input validation OK
[apify.tripadvis

📦 Obtenidas 49 reviews (todas)
✅ 2 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 2, 2: 0, 3: 2, 4: 9, 5: 36}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
❌ Error guardando reviews: UNIQUE constraint failed: Reviews.review_id
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 40 actualizado con histograma
⏰ Esperando 5 segundos...

[2/3] 📖 Escaneando (alternativo): Restaurante Japones Oishi


[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:06.406Z ACTOR: Pulling Docker image of build YjNFzefvG0FzjjRps from registry.
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:06.408Z ACTOR: Creating Docker container.
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:06.637Z ACTOR: Starting Docker container.
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:06.866Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c npm run start:prod --silent
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:09.885Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:XXaXX1fnmYCR7Z9B6] -> 2025-10-04T18:37:10.070Z INFO  Input validation OK
[apify.tripadvis

📦 Obtenidas 80 reviews (todas)
✅ 8 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 5, 2: 3, 3: 17, 4: 35, 5: 20}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ 8 reviews guardadas en la BD
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 1 actualizado con histograma
⏰ Esperando 5 segundos...

[3/3] 📖 Escaneando (alternativo): La Taverna


[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> Status: RUNNING, Message: 
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:27.188Z ACTOR: Pulling Docker image of build YjNFzefvG0FzjjRps from registry.
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:27.189Z ACTOR: Creating Docker container.
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:27.295Z ACTOR: Starting Docker container.
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:27.515Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c npm run start:prod --silent
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:29.933Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.tripadvisor-reviews runId:yD5zH6S0O1JqFYywe] -> 2025-10-04T18:37:30.088Z INFO  Input validation OK
[apify.tripadvis

📦 Obtenidas 111 reviews (todas)
✅ 27 reviews negativas (1-2 estrellas) filtradas
📊 Histograma completo: {1: 19, 2: 8, 3: 16, 4: 38, 5: 30}
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
❌ Error guardando reviews: UNIQUE constraint failed: Reviews.review_id
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Restaurante 2 actualizado con histograma

🎯 ESCANEO COMPLETADO
   • Método usado: alternativo
   • Restaurantes escaneados: 3
   • Reviews negativas encontradas: 37


In [10]:
# Celda adicional: Verificar histograma en la BD
def verificar_histograma():
    """
    Verifica que el histograma se haya guardado correctamente
    """
    conn = get_db_connection()
    
    query = """
    SELECT id, nombre, conteo_rating_1, conteo_rating_2, conteo_rating_3, 
           conteo_rating_4, conteo_rating_5, fecha_escaneo_reviews
    FROM Restaurantes 
    WHERE fecha_escaneo_reviews IS NOT NULL
    ORDER BY fecha_escaneo_reviews DESC
    LIMIT 5
    """
    
    try:
        df = pd.read_sql_query(query, conn)
        print("📊 HISTOGRAMA EN BASE DE DATOS:")
        print("=" * 50)
        if not df.empty:
            for _, row in df.iterrows():
                print(f"🍽️  {row['nombre']}")
                print(f"   ⭐1: {row['conteo_rating_1']} | ⭐2: {row['conteo_rating_2']} | ⭐3: {row['conteo_rating_3']}")
                print(f"   ⭐4: {row['conteo_rating_4']} | ⭐5: {row['conteo_rating_5']}")
                print(f"   📅 Último escaneo: {row['fecha_escaneo_reviews']}")
                print("-" * 30)
        else:
            print("❌ No hay datos de histograma")
            
    except Exception as e:
        print(f"❌ Error verificando histograma: {e}")
    finally:
        conn.close()

# Ejecutar verificación
verificar_histograma()

📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📊 HISTOGRAMA EN BASE DE DATOS:
🍽️  Restaurant Gran Muralla I
   ⭐1: 12 | ⭐2: 2 | ⭐3: 15
   ⭐4: 43 | ⭐5: 28
   📅 Último escaneo: 2025-10-04 18:25:05
------------------------------
🍽️  Can Punxa
   ⭐1: 0 | ⭐2: 0 | ⭐3: 0
   ⭐4: 0 | ⭐5: 0
   📅 Último escaneo: 2025-10-04 18:24:52
------------------------------
🍽️  Terram
   ⭐1: 2 | ⭐2: 1 | ⭐3: 1
   ⭐4: 1 | ⭐5: 24
   📅 Último escaneo: 2025-10-04 18:22:51
------------------------------
🍽️  L’ Origen Girona
   ⭐1: 0 | ⭐2: 0 | ⭐3: 0
   ⭐4: 0 | ⭐5: 0
   📅 Último escaneo: 2025-10-04 18:22:38
------------------------------
🍽️  Indigo Restaurant & Lounge
   ⭐1: 20 | ⭐2: 9 | ⭐3: 37
   ⭐4: 70 | ⭐5: 54
   📅 Último escaneo: 2025-10-04 18:21:00
------------------------------
